<a href="https://colab.research.google.com/github/Vatsaly76/text-to-image/blob/main/Image_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install diffusers transformers accelerate gradio -q

from diffusers import StableDiffusionPipeline
import torch
import gradio as gr
from PIL import Image
import uuid
import os

# Load Stable Diffusion model
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4")
pipe = pipe.to("cuda")

# Ensure image output folder exists
os.makedirs("generated_images", exist_ok=True)

# Image generation function
def generate_image(prompt):
    image = pipe(prompt).images[0]
    filename = f"generated_images/{uuid.uuid4().hex[:8]}.png"
    image.save(filename)
    return image, filename

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.6 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [20]:
with gr.Blocks(css="""
.gradio-container {
    background: linear-gradient(135deg, #0f0c29, #302b63, #24243e);
    padding: 50px 20px;
    min-height: 100vh;
}

/* --- Targeting the h1 tag directly for reliability --- */
#main-title h1 {
    text-align: center;
    font-size: 2rem;
    font-weight: 900;
    margin-bottom: 50px;
    background: -webkit-linear-gradient(45deg, #ffb347, #ffd700);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
}

#promptbox input {
    font-size: 18px;
    padding: 12px;
    border: 2px solid #c800c8;
    border-radius: 10px;
}

#genbtn button {
    background: linear-gradient(45deg, #ff66cc, #cc00ff);
    color: white;
    font-weight: bold;
    font-size: 20px;
    padding: 14px 28px;
    border: none;
    border-radius: 12px;
    margin-top: 10px;
    transition: transform 0.2s ease, box-shadow 0.2s ease;
    box-shadow: 0 4px 12px rgba(0,0,0,0.2);
}
#genbtn button:hover {
    transform: scale(1.05);
    box-shadow: 0 6px 18px rgba(0,0,0,0.25);
}
""") as demo:

    # --- Wrapping the title in an h1 tag ---
    gr.Markdown("<h1>✨ AI Text-to-Image Generator</h1>", elem_id="main-title")

    prompt = gr.Textbox(placeholder="Describe your image...", label="Prompt", elem_id="promptbox")

    btn = gr.Button("🎨 Generate Image", elem_id="genbtn")

    image_output = gr.Image(label="Output Image", type="pil")
    file_output = gr.File(label="Download Image")

    btn.click(fn=run, inputs=prompt, outputs=[image_output, file_output])

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b8e18af55a914d9284.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
